In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import UnitaryGate
from qiskit.providers.aer import AerSimulator

##############################################################################
# 1. Classical Preprocessing: Normalize the adjacency matrix
##############################################################################

# Original weighted adjacency matrix (risk propagation weights)
adjacency_matrix = np.array([[0,    0.10, 0.13],
                             [0.20, 0.0,  0.50],
                             [0.17, 0.30, 0.0]])

# Normalize the outgoing edge weights for each vertex so they sum to 1.
P = np.zeros_like(adjacency_matrix)
for i in range(adjacency_matrix.shape[0]):
    row_sum = np.sum(adjacency_matrix[i])
    if row_sum > 0:
        P[i] = adjacency_matrix[i] / row_sum
    else:
        P[i] = adjacency_matrix[i]

##############################################################################
# 2. Define the coin operator for each vertex
##############################################################################

def coin_operator(i):
    """
    For vertex i, return a 2x2 unitary coin operator that transforms |0> as:
         |0> -> sqrt(p0)|0> + sqrt(p1)|1>
    where p0 and p1 are the normalized probabilities for the two outgoing edges.
    """
    if i == 0:
        p0 = P[0, 1]  # probability to go to vertex 1
        p1 = P[0, 2]  # probability to go to vertex 2
    elif i == 1:
        p0 = P[1, 0]  # probability to go to vertex 0
        p1 = P[1, 2]  # probability to go to vertex 2
    elif i == 2:
        p0 = P[2, 0]  # probability to go to vertex 0
        p1 = P[2, 1]  # probability to go to vertex 1
    # Choose theta so that: cos^2(theta/2)=p0 and sin^2(theta/2)=p1.
    theta = 2 * np.arcsin(np.sqrt(p1))
    return np.array([[np.cos(theta/2), -np.sin(theta/2)],
                     [np.sin(theta/2),  np.cos(theta/2)]], dtype=complex)

# Build the block-diagonal coin operator on the 6-dimensional edge space.
# Ordering:
#   state 0: vertex 0, coin=0 (edge: 0→1)
#   state 1: vertex 0, coin=1 (edge: 0→2)
#   state 2: vertex 1, coin=0 (edge: 1→0)
#   state 3: vertex 1, coin=1 (edge: 1→2)
#   state 4: vertex 2, coin=0 (edge: 2→0)
#   state 5: vertex 2, coin=1 (edge: 2→1)
C = np.zeros((6, 6), dtype=complex)
for i in range(3):
    Ci = coin_operator(i)  # 2x2 unitary for vertex i
    if i == 0:
        C[0:2, 0:2] = Ci
    elif i == 1:
        C[2:4, 2:4] = Ci
    elif i == 2:
        C[4:6, 4:6] = Ci

##############################################################################
# 3. Define the shift operator S on the edge space.
##############################################################################

S = np.zeros((6, 6), dtype=complex)
# Mapping: S maps an edge (i → j) to (j → i)
S[0, 2] = 1.0  # state 0 (edge 0→1)  --> state 2 (edge 1→0)
S[1, 4] = 1.0  # state 1 (edge 0→2)  --> state 4 (edge 2→0)
S[2, 0] = 1.0  # state 2 (edge 1→0)  --> state 0 (edge 0→1)
S[3, 5] = 1.0  # state 3 (edge 1→2)  --> state 5 (edge 2→1)
S[4, 1] = 1.0  # state 4 (edge 2→0)  --> state 1 (edge 0→2)
S[5, 3] = 1.0  # state 5 (edge 2→1)  --> state 3 (edge 1→2)

##############################################################################
# 4. Define one step of the quantum walk: U_qw = S * C
##############################################################################

U_qw = S @ C  # This is a 6x6 unitary

##############################################################################
# 5. Embed the 6-dimensional unitary into an 8-dimensional unitary (for 3 qubits)
##############################################################################

U_embed = np.eye(8, dtype=complex)
U_embed[:6, :6] = U_qw  # Leave the extra two basis states unchanged

# Create a UnitaryGate using the updated Qiskit import.
qw_gate = UnitaryGate(U_embed, label="QW_Step")

##############################################################################
# 6. Build the Qiskit circuit with multiple steps of the quantum walk
##############################################################################

# We use 3 qubits to represent the 8-dimensional Hilbert space.
qr = QuantumRegister(3, 'qw')
cr = ClassicalRegister(3, 'c')
qc = QuantumCircuit(qr, cr)

# --- Initial State Preparation ---
# Encode the initial risk vector into the vertices.
# For vertex i with risk r_i, we distribute the amplitude equally over its two coin states.
risk_vector = np.array([0.2, 0.3, 0.5])
initial_state = np.zeros(8, dtype=complex)
# Mapping:
#   state 0: vertex 0, coin=0
#   state 1: vertex 0, coin=1
#   state 2: vertex 1, coin=0
#   state 3: vertex 1, coin=1
#   state 4: vertex 2, coin=0
#   state 5: vertex 2, coin=1
initial_state[0] = np.sqrt(risk_vector[0]) / np.sqrt(2)
initial_state[1] = np.sqrt(risk_vector[0]) / np.sqrt(2)
initial_state[2] = np.sqrt(risk_vector[1]) / np.sqrt(2)
initial_state[3] = np.sqrt(risk_vector[1]) / np.sqrt(2)
initial_state[4] = np.sqrt(risk_vector[2]) / np.sqrt(2)
initial_state[5] = np.sqrt(risk_vector[2]) / np.sqrt(2)
# States 6 and 7 remain 0 (unused)

# Initialize the circuit state.
qc.initialize(initial_state, qr)
qc.barrier()

# --- Apply multiple steps of the quantum walk ---
num_steps = 3
for step in range(num_steps):
    qc.append(qw_gate, qr)
    qc.barrier()

# Make a copy of the circuit without measurements for statevector simulation.
qc_no_meas = qc.copy()

# Now add measurements to a copy for shot-based simulation.
qc_meas = qc.copy()
qc_meas.measure(qr, cr)

##############################################################################
# 7. Compute the Expected Final State via Matrix Multiplication
##############################################################################

# The overall unitary for num_steps is U_embed^(num_steps)
expected_state = np.linalg.matrix_power(U_embed, num_steps) @ initial_state

##############################################################################
# 8. Simulate the Circuit (Statevector Simulation)
##############################################################################

sim_sv = AerSimulator(method="statevector")
result_sv = sim_sv.run(qc_no_meas).result()
final_statevector = result_sv.get_statevector(qc_no_meas)

print("Final statevector from simulation (no measurements) after {} steps:".format(num_steps))
print(final_statevector)
print("\nExpected statevector computed classically:")
print(expected_state)
difference_norm = np.linalg.norm(final_statevector - expected_state)
print("\nNorm of the difference between simulated and expected statevector:", difference_norm)

##############################################################################
# 9. Simulate the Circuit (QASM Simulation for Measurement Counts)
##############################################################################

sim_qasm = AerSimulator(method="qasm")
result_qasm = sim_qasm.run(qc_meas, shots=1024).result()
counts = result_qasm.get_counts(qc_meas)
print("\nMeasurement counts (over 1024 shots):")
print(counts)

##############################################################################
# 10. (Optional) Draw the circuit for visualization
##############################################################################

print("\nCircuit:")
print(qc.draw(output='text'))



ImportError: cannot import name 'execute' from 'qiskit' (/Users/user/Documents/GitHub/Quantum-SWE-Project/.venv/lib/python3.12/site-packages/qiskit/__init__.py)